In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [ ]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Twitter Sentiment Analysis") \
    .getOrCreate()

# Define the schema
schema = StructType([
    StructField("target", IntegerType(), True),
    StructField("id", StringType(), True),
    StructField("date", StringType(), True),
    StructField("flag", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True)
])

# Read the CSV file with the defined schema
df = spark.read.csv("training.1600000.processed.noemoticon.csv", schema=schema, header=False, inferSchema=False)

In [ ]:
df_clean = df.withColumn("text", regexp_replace(col("text"), "@\w+", "")) \
.withColumn("text", regexp_replace(col("text"), "http\S+", "")) \
.withColumn("text", regexp_replace(col("text"), "[^a-zA-Z\s]", ""))

In [ ]:
# Feature Engineering and Model Training

tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
vectorizer = CountVectorizer(inputCol="filtered", outputCol="features")
idf = IDF(inputCol="features", outputCol="idf_features")
classifier = LogisticRegression(featuresCol="idf_features", labelCol="label")

pipeline = Pipeline(stages=[tokenizer, remover, vectorizer, idf, classifier])
model = pipeline.fit(df_clean)

In [ ]:
# Prediction and Evaluation

predictions = model.transform(test_data)
predictions.select("text", "prediction").show()